In [1]:
import Preprocessing
import pandas as pd

In [2]:
PATH = "./data/"
ds = pd.read_csv(PATH + "train.csv")
ds_test = pd.read_csv(PATH + "test.csv")
ds_season = pd.read_csv(PATH + "season.csv")

### 1 刪除缺項過多feature，刪除完全沒有資料的人

In [3]:
ds, ds_test = Preprocessing.RemoveData(ds, ds_test, cols=['最高學歷', '畢業學校類別'])

### 2 season合併
加班數,出差數A,出差數B,請假數A,請假數B, 出差數A+B, 請假數A+B

In [4]:
train_merged, test_merged = Preprocessing.mergeSeason(ds, ds_test, ds_season)

/homes/ssd3/weitung/.conda/envs/test/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


### Train AB加總
訓練時數A,訓練時數B,訓練時數C, <br>
近三月請假數A,近三月請假數B, <br>
近一年請假數A,近一年請假數B, <br>
出差數A,出差數B, <br>
年度績效等級A,年度績效等級B,年度績效等級C, <br>
年資層級A,年資層級B,年資層級C <br>
--> 新增feat：訓練時數A+B+C, 近三月請假數A+B, 近一年請假數A+B, 出差數A+B, 年度績效等級A+B+C, 年資層級A+B+C

In [5]:
ds_train_all = Preprocessing.TrainAdd(train_merged)
ds_test_all = Preprocessing.TrainAdd(test_merged)

### Normalize for num

### 補0

### Feature Selection
1. Decomposition<br>
2. Percent（與原本那3個方法相似）<br>
3. 原本那3個<br>

### 生csv

In [6]:
X_train, y_train = Preprocessing.splitXY(ds_train_all)
X_test, y_test = Preprocessing.splitXY(ds_test_all)
y_train.to_csv('y_train.csv', index=False)

In [7]:
selectors = ['Mutual', 'Chi', 'ANOVA', 'NMF', 'PCA']
for way in ["maxmin", "std"]:
    print(way)
    if way == 'maxmin':
        train, test = Preprocessing.maxminNormalize(X_train, X_test, Preprocessing.NUM_FEATS)
        test.fillna(0, inplace=True)
        for i in range(5):
            print(selectors[i])
            name = "./data/" + way + "_" + selectors[i]
            if i == 0:
                x_train, x_test, _ = Preprocessing.keepMutual(train, y_train, test, k=10)
            elif i == 1:
                x_train, x_test, _ = Preprocessing.keepChi(train, y_train, test, k=10)
            elif i == 2:
                x_train, x_test, _ = Preprocessing.keepANOVA(train, y_train, test, k=10)
            elif i == 3:
                x_train, x_test = Preprocessing.keepNMF(train, test, k=10)
            elif i == 4:
                x_train, x_test = Preprocessing.keepPCA(train, test, k=10)
                
            x_train.to_csv(name + '_x_train.csv', index=False)
            x_test.to_csv(name + '_x_test.csv', index=False)
    else:
        train, test = Preprocessing.stdNormalize(X_train, X_test, Preprocessing.NUM_FEATS)
        test.fillna(0, inplace=True)
        for i in range(0, 5, 2): # skip chi & NMF
            print(selectors[i])
            name = "./data/" + way + "_" + selectors[i]
            if i == 0:
                x_train, x_test, _ = Preprocessing.keepMutual(train, y_train, test, k=10)
            elif i == 2:
                x_train, x_test, _ = Preprocessing.keepANOVA(train, y_train, test, k=10)
            elif i == 4:
                x_train, x_test = Preprocessing.keepPCA(train, test, k=10)
                
            x_train.to_csv(name + '_x_train.csv', index=False)
            x_test.to_csv(name + '_x_test.csv', index=False)

maxmin
Mutual
Chi
ANOVA
NMF


/homes/ssd3/weitung/.conda/envs/test/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1692: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


PCA
std
Mutual
ANOVA
PCA
